In [1]:
#Punto 1
from fractions import Fraction


a = Fraction(1, 2)
b = Fraction(2, 3)
print(a + b, a * b)

7/6 1/3


In [2]:
class Corpo: #Punto 2
    def __init__(self, pos):
        self.posizione = Fraction(pos)  #converto input in frazione prima di salvarlo

    def muoviti_per_t(self, tempo): #Punto 6
        self.posizione = self.posizione + self.velocità() * Fraction(tempo)

    def __lt__(self, other): #Punto 9
        if ((self.posizione < other.posizione)
                or (self.posizione == other.posizione
                    and self.velocità() < other.velocità())):
            return True
        return False

    def __le__(self, other): #Punto 9
        if ((self.posizione < other.posizione)
                or (self.posizione == other.posizione
                    and self.velocità() <= other.velocità())):
            return True
        return False

    def kin(self): #Punto 12
        if self.mobile()==False:
            return Fraction(0)
        return Fraction(0.5)*self.m*self.velocità()*self.velocità()

In [3]:
m = Corpo(2)

In [4]:
type(m)

__main__.Corpo

In [5]:
m.posizione

Fraction(2, 1)

In [6]:
m.posizione + b

Fraction(8, 3)

In [7]:
class Muro(Corpo): #Punto 3
    def mobile(self):
        return False

    def __repr__(self):
        rep = 'Muro({})'.format(self.posizione)
        return rep

    def velocità(self):
        return Fraction(0)

In [8]:
wall = Muro(0)
wall

Muro(0)

In [9]:
wall.kin()

Fraction(0, 1)

In [10]:
class Carrello(Corpo): #Punto 4
    def __init__(self, pos_carrello, m_carrello, v_carrello):
        self.posizione = pos_carrello
        Corpo.__init__(self, self.posizione)
        self.m = Fraction(m_carrello)
        self.v = Fraction(v_carrello)

    def velocità(self):
        return self.v

    def cambio_v(self, nuova_v): #Punto 5: metodo "imposta velocità"
        self.v = nuova_v

    def mobile(self):
        return True
    
    def __repr__(self):
        rep2 = 'Carrello(x:{},m:{},v:{})'.format(self.posizione,self.m, self.v)
        return rep2

In [11]:
cart = Carrello(2, 3, 4)

In [12]:
cart.velocità()

Fraction(4, 1)

In [13]:
cart.kin()

Fraction(24, 1)

In [14]:
wall.velocità()

Fraction(0, 1)

In [15]:
cart.posizione

Fraction(2, 1)

In [16]:
cart.muoviti_per_t(2)
cart

Carrello(x:10,m:3,v:4)

In [18]:
class Rotaia: #Punto 7
    def __init__(self):
        self.lista = []

    def aggiungi_corpo(self, crp): #Punto 8
        self.lista.append(crp)
        self.lista.sort()

    def prossimo_urto(self): #Punto 10
        lista_tempi = []
        for i in range(len(self.lista)-1):
            #se velocità a dx è >= di quella a sx, l'oggetto i e l'oggetto i+1 non si toccheranno mai (il ciclo salta l'ultimo oggetto)
            if (self.lista[i+1].velocità() < self.lista[i].velocità()):
                t = (self.lista[i+1].posizione - self.lista[i].posizione)/(self.lista[i].velocità() - self.lista[i+1].velocità())
                lista_tempi.append(t)
        if (lista_tempi == []):
            return None
        return min(lista_tempi)

    def kin_totale(self): #Punto 12
        kin_total = 0
        for i in range(0,len(self.lista)):
            kin_total = kin_total + self.lista[i].kin()
        return kin_total

    def urti(self): #Punto 11
        n_urti = 0

        while (self.prossimo_urto() != None):
            t_evo = self.prossimo_urto()

            #aggiorno posizioni
            for i  in range(len(self.lista)):
                self.lista[i].muoviti_per_t(t_evo)
            for ii in range(len(self.lista)-2):
                if self.lista[ii].posizione == self.lista[ii+1].posizione == self.lista[ii+2].posizione:
                    raise Exception("Can't handle urti a 3 corpi")
            #aggiorno velocità
            for j in  range(len(self.lista)-1):
                if self.lista[j].posizione == self.lista[j+1].posizione:
                    n_urti += 1
                    #se hanno il tipo diverso uno è un muro
                    if type(self.lista[j]) != type(self.lista[j+1]):
                        #i è il muro, i+1 è il carrello
                        if   self.lista[j  ].mobile() == False:
                             self.lista[j+1].cambio_v(-self.lista[j+1].velocità())
                        #i è il carrello, i+1 è il muro
                        elif self.lista[j+1].mobile() == False:
                             self.lista[j  ].cambio_v(-self.lista[j  ].velocità())
                        #ho due carrelli
                    else:
                        #cambio velocità carrello j
                        old_speed1 = self.lista[j  ].velocità() #salvo la vecchia velocità del carrello a sx, per poter aggiornare quella del carrello a dx
                        self.lista[j  ].cambio_v(  (self.lista[j  ].m - self.lista[j+1].m)/(self.lista[j  ].m + self.lista[j+1].m)*self.lista[j  ].velocità()   +
                                                                2*self.lista[j+1].m /(self.lista[j  ].m + self.lista[j+1].m)*self.lista[j+1].velocità()  )
                        #cambio velocità carrello j+1
                        self.lista[j+1].cambio_v(  (self.lista[j+1].m - self.lista[j  ].m)/(self.lista[j  ].m + self.lista[j+1].m)*self.lista[j+1].velocità()   +
                                                                2*self.lista[j  ].m /(self.lista[j  ].m + self.lista[j+1].m)*old_speed1  )
        return n_urti

In [19]:
trail = Rotaia()

In [20]:
cart2 = Carrello(4, 1, -2)

In [21]:
trail.aggiungi_corpo(cart2)


In [22]:
print(trail.lista)

[Carrello(x:4,m:1,v:-2)]


In [23]:
cart3 = Carrello(6, 3, -6)


In [24]:
trail.aggiungi_corpo(cart3)

In [25]:
trail.aggiungi_corpo(wall)

In [26]:
wall <= cart

True

In [27]:
print(trail.lista)

[Muro(0), Carrello(x:4,m:1,v:-2), Carrello(x:6,m:3,v:-6)]


In [28]:
trail.prossimo_urto()

Fraction(1, 2)

In [29]:
trail = Rotaia()
cart2 = Carrello(1, 1, -1)
trail.aggiungi_corpo(cart2)
print(trail.lista)
trail.aggiungi_corpo(wall)

cart3 = Carrello(2, 1, 0)
trail.aggiungi_corpo(cart3)

cart4 = Carrello(3, 1, 0)
trail.aggiungi_corpo(cart4)

print(trail.lista)
trail.kin_totale()

[Carrello(x:1,m:1,v:-1)]
[Muro(0), Carrello(x:1,m:1,v:-1), Carrello(x:2,m:1,v:0), Carrello(x:3,m:1,v:0)]


Fraction(1, 2)

In [30]:
#Punto 13
n1n2_trail = Rotaia() 
m_carrelli = 1
v_carrelli = 2
n1 = 4
n2 = 6
dist_ij = 1
dist_ii = 1
dist_jj = 1
[n1n2_trail.aggiungi_corpo(Carrello(dist_ii*i                                             ,m_carrelli, v_carrelli)) for i in range(0,n1)]
[n1n2_trail.aggiungi_corpo(Carrello(n1n2_trail.lista[n1-1].posizione + dist_ij + dist_jj*j,m_carrelli,-v_carrelli)) for j in range(0,n2)]
print('Rotaia:', n1n2_trail.lista)
print("en kin iniziale:", n1n2_trail.kin_totale()) #Punto 12
n_urti = n1n2_trail.urti()
print("en kin finale:", n1n2_trail.kin_totale()) #Punto 12
print('numero totale urti:', n_urti, ", numero teorico:", n1*n2) #il numero teorico è proprio quello perchè ogni urto equivale a una trasposizione tra due carrelli

Rotaia: [Carrello(x:0,m:1,v:2), Carrello(x:1,m:1,v:2), Carrello(x:2,m:1,v:2), Carrello(x:3,m:1,v:2), Carrello(x:4,m:1,v:-2), Carrello(x:5,m:1,v:-2), Carrello(x:6,m:1,v:-2), Carrello(x:7,m:1,v:-2), Carrello(x:8,m:1,v:-2), Carrello(x:9,m:1,v:-2)]
en kin iniziale: 20
en kin finale: 20
numero totale urti: 24 , numero teorico: 24


In [34]:
#Punto 14
pi_trail = Rotaia()
pi_muro      = Muro(0)
pi_carrello1 = Carrello(2, 1 , 0)
pi_trail.aggiungi_corpo(pi_muro)
pi_trail.aggiungi_corpo(pi_carrello1)

cifre_pi     = 4
m_cart2      = 10**(2*(cifre_pi-1))
pi_carrello2 = Carrello(10, m_cart2, -2)
pi_trail.aggiungi_corpo(pi_carrello2)

print('Rotaia:', pi_trail.lista)
n_urti_pi = pi_trail.urti()
pigreco = str(n_urti_pi)
print("prime",cifre_pi,"cifre di pigreco:", pigreco[0]+"."+pigreco[1:cifre_pi])

Rotaia: [Muro(0), Carrello(x:2,m:1,v:0), Carrello(x:10,m:1000000,v:-2)]


prime 4 cifre di pigreco: 3.141
